## Cache

In [11]:
import os
import langchain
from langchain_openai import OpenAI
from langchain.cache import InMemoryCache
from dotenv import load_dotenv

In [12]:
langchain.llm_cache = InMemoryCache()
load_dotenv()
open_api_key = os.getenv("OPEN_API_KEY")

In [14]:
# Enable in-memory caching
langchain.llm_cache = InMemoryCache()

# Initialize an LLM (replace with your API key if needed)
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0, api_key=open_api_key)

# First call → will query the API
response1 = llm.invoke("Tell me a joke about LangChain")
print("First call:", response1)

# Second call with the same prompt → served from cache
response2 = llm.invoke("Tell me a joke about LangChain")
print("Second call (cached):", response2)

First call: 

Why did the programming language go on a diet?

Because it wanted to be a lean, mean LangChain machine!
Second call (cached): 

Why did the programming language go on a diet?

Because it wanted to be a lean, mean LangChain machine!


## Prompttemplate
* Purpose: Used for text-based LLMs (completion models).
* Output: A single string prompt.
* Use case: When you want to generate a plain text prompt (no roles like “system” or “user”).

In [1]:
from langchain.prompts import PromptTemplate

# Single input variable: "product"
single_input_template = PromptTemplate(
    input_variables=["product"],
    template="What are three benefits of using {product}?"
)

# Format with input
formatted_prompt = single_input_template.format(product="LangChain")
print(formatted_prompt)

What are three benefits of using LangChain?


In [3]:
from langchain.prompts import PromptTemplate

# Multiple input variables: "adjective" and "product"
multi_input_template = PromptTemplate(
    input_variables=["adjective", "product"],
    template="Write a {adjective} review about {product}."
)

# Format with inputs
formatted_prompt = multi_input_template.format(adjective="positive", product="LangChain")
print(formatted_prompt)

Write a positive review about LangChain.


## ChatPromptTemplate
* Purpose: Used for chat-based LLMs (like OpenAI’s GPT-4, GPT-4o, GPT-3.5).
* Output: A list of message objects (with roles: system, human, ai, etc.).
* Use case: When you need structured multi-role prompts (system instructions + user queries).

In [15]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [16]:
load_dotenv()
open_api_key = os.getenv("OPEN_API_KEY")

In [17]:
# Document as a string
document_text = """
LangChain is a framework for developing applications powered by language models.
It provides abstractions for prompts, chains, agents, memory, and integrates with many external tools.
"""

# Define the chat prompt template with system + human messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the provided document to answer questions."),
    ("human", "Here is the document:\n{document}\n\nQuestion: {question}")
])

# Format the prompt with input variables
messages = chat_prompt.format_messages(
    document=document_text,
    question="What does LangChain provide?"
)

# Initialize the model
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=open_api_key)

# Get the response
response = chat.invoke(messages)
print(response.content)


LangChain provides abstractions for prompts, chains, agents, memory, and integrates with many external tools.


In [19]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [20]:
# Document string
document_text = """
LangChain is a framework for developing applications powered by language models.
It provides abstractions for prompts, chains, agents, memory, and integrates with many external tools.
"""

# Create system and human message prompt templates
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant. Use the provided document to answer questions."
)

human_prompt = HumanMessagePromptTemplate.from_template(
    "Here is the document:\n{document}\n\nQuestion: {question}"
)

# Build the chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

# Format with inputs
messages = chat_prompt.format_messages(
    document=document_text,
    question="What does LangChain provide?"
)

# Initialize LLM
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0,  api_key=open_api_key)

# Invoke the model
response = chat.invoke(messages)
print(response.content)

LangChain provides abstractions for prompts, chains, agents, memory, and integrates with many external tools.


## Few Shots Prompt-Template

### The Rule Detective: A Few-Shot Learning Example with LangChain
The "Rule Detective" game is a perfect, simplified metaphor for few-shot learning. The "Rule Master" acts as the data provider, giving a few examples, and the "Rule Detective" acts as the language model, inferring the underlying pattern.

Here's how to structure this game's logic using LangChain's prompt templates.

1. The System Prompt: The "Game Rules"
The SystemMessagePromptTemplate establishes the persona and core instructions for the "model." In our game, this sets the stage for the AI's role and the objective.

Prompt Template Logic: The system message tells the AI its identity and the rules it must follow to be a successful "Rule Detective."

In [3]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import random

In [4]:
def play_rule_master_game():
    """
    This game simulates the "Rule Master" for a few-shot learning demonstration.
    The user acts as the "Rule Detective" to guess a hidden rule based on examples.
    """
    # 1. Define the possible rules and their examples
    rules_data = [
        {
            "rule": "All items are round.",
            "examples": ["ball", "orange", "moon"],
            "non_examples": ["book", "chair", "pencil"]
        },
        {
            "rule": "All items can be found in a kitchen.",
            "examples": ["spoon", "bowl", "refrigerator"],
            "non_examples": ["car", "tree", "shoe"]
        },
        {
            "rule": "All items are made of metal.",
            "examples": ["coin", "key", "wrench"],
            "non_examples": ["grape", "plastic bottle", "cloud"]
        }
    ]

    # 2. Randomly select a secret rule for the game
    secret_rule_data = random.choice(rules_data)
    secret_rule = secret_rule_data["rule"]
    few_shot_examples = secret_rule_data["examples"]
    
    print("Welcome to The Rule Master Game!")
    print("I have a secret rule in mind. Your job is to figure it out.")
    print("-" * 40)
    
    # 3. Present the System Prompt (The Game Rules)
    system_message = SystemMessagePromptTemplate.from_template(
        "You are an expert logician and a brilliant detective. Your task is to identify a secret rule that I have in my mind. I will give you a few examples of items that follow this rule, and then I will ask you about other items. Your final goal is to guess the rule based on the examples and our conversation."
    )
    print(f"**System Prompt:**\n{system_message.prompt.template}\n")

    # 4. Present the Few-Shot Examples (The Training Data)
    print("**Few-Shot Examples:**")
    for item in few_shot_examples:
        human_message = HumanMessagePromptTemplate.from_template(f"A **{item}** follows the rule.")
        print(f" - {human_message.prompt.template}")
    print("\n" + "-" * 40)

    # 5. The Conversation Loop (Inference)
    print("Let the guessing begin!")
    print("Type an item to ask if it follows the rule. Type 'guess' to make your final guess.")
    
    while True:
        user_input = input("\nYour item or action: ").strip().lower()

        if user_input == 'guess':
            final_guess = input("What is your final guess for the rule? ")
            if final_guess.strip().lower() == secret_rule.strip().lower():
                print("\nCongratulations! You are a master detective! You guessed the rule correctly.")
            else:
                print("\nThat's a good guess, but it's not the secret rule. The rule was:")
                print(f"**{secret_rule}**")
            break

        # Check if the user's item follows the rule
        if user_input in secret_rule_data["examples"] or user_input in secret_rule_data["non_examples"]:
            if user_input in secret_rule_data["examples"]:
                print(f"My answer is: Yes, a **{user_input}** follows the rule.")
            else:
                print(f"My answer is: No, a **{user_input}** does not follow the rule.")
        else:
            print("That item is not in my knowledge base. Please try another one.")


In [5]:
play_rule_master_game()

Welcome to The Rule Master Game!
I have a secret rule in mind. Your job is to figure it out.
----------------------------------------
**System Prompt:**
You are an expert logician and a brilliant detective. Your task is to identify a secret rule that I have in my mind. I will give you a few examples of items that follow this rule, and then I will ask you about other items. Your final goal is to guess the rule based on the examples and our conversation.

**Few-Shot Examples:**
 - A **ball** follows the rule.
 - A **orange** follows the rule.
 - A **moon** follows the rule.

----------------------------------------
Let the guessing begin!
Type an item to ask if it follows the rule. Type 'guess' to make your final guess.



Your item or action:  ball, orange, moon


That item is not in my knowledge base. Please try another one.


KeyboardInterrupt: Interrupted by user